In [ ]:
#!/usr/bin/python

import socket
import sys
import re
import time
import psycopg2
import RPi.GPIO as GPIO



import psycopg2

from datetime import datetime, timezone
from AtlasI2C import AtlasI2C

GPIO_PIN = 25  # This is the pin for the solenoid valve
PUMP_PIN = 24  # This is the pin for the pump

# Set up GPIO
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    for address in dev.list_i2c_devices():
        dev.set_i2c_address(address)
        if 11 <= address <= 127:
            moduletype = "ERR"
            while moduletype == "ERR":
                response = dev.query("I")
                if ("HUM" in response):
                    moduletype = "HUM"
                elif ("CO2" in response):
                    moduletype = "CO2"
                elif ("PRS" in response):
                    moduletype = "PRS"
                else:
                    error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)
            name = "UNKNOWN"
            while (name == "UNKNOWN") or (bool == False):
                bool = False 
                response = dev.query("name,?")
                if ("NAME," in response):
                    name = response.split(",")[1]
                    name = name[:9]
                    r = re.compile('.{3}-.{2}-.{2}')
                    if r.match(name):
                        bool = True
                else:
                    error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)

            device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

    for device in device_list:
        print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    return device_list

def setProperties(device_list):
    for device in device_list:
        if (device.moduletype == "HUM"):
            device.write("O,T,1")
            time.sleep(1)
            device.write("O,Dew,1")
            time.sleep(1)
        if (device.moduletype == "CO2"):
            device.write("O,t,1")
            time.sleep(1)

def writeHeader(output_file, device_list):
    csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
    output_file.write(csv_header+"\n")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        device_output = device.read()
        device_output.encode('ascii')
        device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
    except:
        print("Reading I2C device failed: "+device.name)
        return reading, str(device_output)

    if (device.moduletype == "HUM"):
        try:
            data = device_output.split(': ')[1]
            hum = float(data.split(',')[0])
            temp = float(data.split(',')[1])
            dew = float(data.split(',')[3])
        except:
            hum = 0.0
            temp = 0.0
            dew = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device:" 
        reading += str(hum) + ", "
        reading += str(temp) + ", "
        reading += str(dew)
    if (device.moduletype == "CO2"):
        try:
            data = device_output.split(': ')[1]
            co2 = float(data.split(',')[0])
            temp = float(data.split(',')[1])
        except:
            co2 = 0.0
            temp = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(co2) + ", "
        reading += str(temp)
    if (device.moduletype == "PRS"):
        try:
            data = device_output.split(': ')[1]
            prs = data
        except:
            prs = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(prs)

    return reading, str(device_output), errMsg

def main():
    host = "unemit-research.postgres.database.azure.com"
    dbname = "sensor_data"
    user = "clerk"
    password = "A*shht6Qy5&GQl29"
    sslmode = "require"
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string) 
    print("Database connection established")
    cursor = conn.cursor()
    
    device_list = detectSensors()    
    if (not device_list):
        sys.stderr.write("No sensors detected. Quitting...\n")
        sys.exit(1)

    setProperties(device_list)

    output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
    header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
        error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    with open(output_filename, "w", buffering=1) as output_file:
        writeHeader(output_file, device_list)
        while True:
            requestReading(device_list)
            time.sleep(1)
            for device in device_list:
                currentReading, probeString, errorMessage = pollDevice(device)
                data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                date_time = datetime.now()
                data += ", "+str(socket.gethostname()) + ", " + device.name
                if (device.moduletype == "HUM"):
                    data1 = data + ", humidity, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", temperature," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    data3 = data + ", dew point," + currentReading.split(',')[2]
                    output_file.write(str(data3)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "humidity", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "temperature", currentReading.split(',')[1], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "dew point", currentReading.split(',')[2], \
                        errorMessage, probeString))                    
                                            
                if (device.moduletype == "CO2"):
                    data1 = data + ", CO2, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", sensor temp," + currentReading.split(',')[1]
                    data2 = data + ", sensor temp," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "CO2", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "sensor temp", currentReading.split(',')[1], \
                        errorMessage, probeString))  
                                         
                if (device.moduletype == "PRS"):
                    data1 = data + ", pressure, " + currentReading
                    output_file.write(str(data1)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "pressure", currentReading, \
                        errorMessage, probeString))                 
                conn.commit()
            time.sleep(10)
    
    
def open_solenoid_valve():
    GPIO.output(GPIO_PIN, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(GPIO_PIN, GPIO.LOW)

def turn_pump_on():
    GPIO.output(PUMP_PIN, GPIO.HIGH)

def turn_pump_off():
    GPIO.output(PUMP_PIN, GPIO.LOW)

def read_co2_level():
    device = AtlasI2C()
    device.set_i2c_address(0x61)  # Assuming the CO2 sensor address is 0x61
    response = device.query("R")
    co2_level = float(response.split(":")[1])
    return co2_level

def connect_to_database():
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string) 
    print("Database connection established")
    return conn

def main():
    while True:
        print("\n--- Menu ---")
        print("1. Open solenoid valve for 1 sec")
        print("3. Turn pump on")
        print("4. Turn pump off")
        print("5. Quit")
        choice = input("Enter your choice: ")
        
        if choice == "1":
            open_solenoid_valve()
        elif choice == "2":
            close_solenoid_valve()
        elif choice == "3":
            turn_pump_on()
        elif choice == "4":
            turn_pump_off()
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 7.")

if __name__ == '__main__':
    main()


In [ ]:
#!/usr/bin/python

import socket
import sys
import re
import time
import psycopg2
import RPi.GPIO as GPIO



import psycopg2

from datetime import datetime, timezone
from AtlasI2C import AtlasI2C

GPIO_PIN = 25  # This is the pin for the solenoid valve
PUMP_PIN = 24  # This is the pin for the pump

# Set up GPIO
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    for address in dev.list_i2c_devices():
        dev.set_i2c_address(address)
        if 11 <= address <= 127:
            moduletype = "ERR"
            while moduletype == "ERR":
                response = dev.query("I")
                if ("HUM" in response):
                    moduletype = "HUM"
                elif ("CO2" in response):
                    moduletype = "CO2"
                elif ("PRS" in response):
                    moduletype = "PRS"
                else:
                    error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)
            name = "UNKNOWN"
            while (name == "UNKNOWN") or (bool == False):
                bool = False 
                response = dev.query("name,?")
                if ("NAME," in response):
                    name = response.split(",")[1]
                    name = name[:9]
                    r = re.compile('.{3}-.{2}-.{2}')
                    if r.match(name):
                        bool = True
                else:
                    error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)

            device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

    for device in device_list:
        print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    return device_list

def setProperties(device_list):
    for device in device_list:
        if (device.moduletype == "HUM"):
            device.write("O,T,1")
            time.sleep(1)
            device.write("O,Dew,1")
            time.sleep(1)
        if (device.moduletype == "CO2"):
            device.write("O,t,1")
            time.sleep(1)

def writeHeader(output_file, device_list):
    csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
    output_file.write(csv_header+"\n")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        device_output = device.read()
        device_output.encode('ascii')
        device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
    except:
        print("Reading I2C device failed: "+device.name)
        return reading, str(device_output)

    if (device.moduletype == "HUM"):
        try:
            data = device_output.split(': ')[1]
            hum = float(data.split(',')[0])
            temp = float(data.split(',')[1])
            dew = float(data.split(',')[3])
        except:
            hum = 0.0
            temp = 0.0
            dew = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device:" 
        reading += str(hum) + ", "
        reading += str(temp) + ", "
        reading += str(dew)
    if (device.moduletype == "CO2"):
        try:
            data = device_output.split(': ')[1]
            co2 = float(data.split(',')[0])
            temp = float(data.split(',')[1])
        except:
            co2 = 0.0
            temp = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(co2) + ", "
        reading += str(temp)
    if (device.moduletype == "PRS"):
        try:
            data = device_output.split(': ')[1]
            prs = data
        except:
            prs = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(prs)

    return reading, str(device_output), errMsg

def main():
    host = "unemit-research.postgres.database.azure.com"
    dbname = "sensor_data"
    user = "clerk"
    password = "A*shht6Qy5&GQl29"
    sslmode = "require"
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string) 
    print("Database connection established")
    cursor = conn.cursor()
    
    device_list = detectSensors()    
    if (not device_list):
        sys.stderr.write("No sensors detected. Quitting...\n")
        sys.exit(1)

    setProperties(device_list)

    output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
    header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
        error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    with open(output_filename, "w", buffering=1) as output_file:
        writeHeader(output_file, device_list)
        while True:
            requestReading(device_list)
            time.sleep(1)
            for device in device_list:
                currentReading, probeString, errorMessage = pollDevice(device)
                data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                date_time = datetime.now()
                data += ", "+str(socket.gethostname()) + ", " + device.name
                if (device.moduletype == "HUM"):
                    data1 = data + ", humidity, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", temperature," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    data3 = data + ", dew point," + currentReading.split(',')[2]
                    output_file.write(str(data3)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "humidity", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "temperature", currentReading.split(',')[1], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "dew point", currentReading.split(',')[2], \
                        errorMessage, probeString))                    
                                            
                if (device.moduletype == "CO2"):
                    data1 = data + ", CO2, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", sensor temp," + currentReading.split(',')[1]
                    data2 = data + ", sensor temp," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "CO2", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "sensor temp", currentReading.split(',')[1], \
                        errorMessage, probeString))  
                                         
                if (device.moduletype == "PRS"):
                    data1 = data + ", pressure, " + currentReading
                    output_file.write(str(data1)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "pressure", currentReading, \
                        errorMessage, probeString))                 
                conn.commit()
            time.sleep(10)
    
    
def open_solenoid_valve():
    GPIO.output(GPIO_PIN, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(GPIO_PIN, GPIO.LOW)

def turn_pump_on():
    GPIO.output(PUMP_PIN, GPIO.HIGH)

def turn_pump_off():
    GPIO.output(PUMP_PIN, GPIO.LOW)

def read_co2_level():
    device = AtlasI2C()
    device.set_i2c_address(0x61)  # Assuming the CO2 sensor address is 0x61
    response = device.query("R")
    co2_level = float(response.split(":")[1])
    return co2_level

def connect_to_database():
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string) 
    print("Database connection established")
    return conn

def main():
    while True:
        print("\n--- Menu ---")
        print("1. Open solenoid valve for 1 sec")
        print("3. Turn pump on")
        print("4. Turn pump off")
        print("5. Quit")
        choice = input("Enter your choice: ")
        
        if choice == "1":
            open_solenoid_valve()
        elif choice == "2":
            close_solenoid_valve()
        elif choice == "3":
            turn_pump_on()
        elif choice == "4":
            turn_pump_off()
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 7.")

if __name__ == '__main__':
    main()


# Manual Code re do! 

In [ ]:
try:
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
    from multiprocessing import Process
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

# GPIO setup
GPIO_PIN = 25
PUMP_PIN = 24
PUMP_PIN_2 = 23
HEATER_PIN = 4

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN_2, GPIO.OUT)
GPIO.setup(HEATER_PIN, GPIO.OUT)

def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue
            
def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        return "", "", "Unexpected error"

# CONSTANTS

N = 10 

#HUM_THRESHOLD_MAX = 20 # % RH
#HUM_THRESHOLD_MIN = 10 # % RH
#TEMP_THRESHOLD = 25 # deg C
#CO2_THRESHOLD = 400

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)


PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 10
HEATER_ON_DURATION = 60
WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUM_CONTROL_INTERVAL = 45
TEMP_CONTROL_INTERVAL = 155

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

# ~~~~
# DEFINITIONS       
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()

# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.05)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_on_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_on_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_off_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_off_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_on_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
        time.sleep(HEATER_ON_DURATION)
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_off_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

#~~~~~~~~

def co2_control_action():
    print("Opening CO2 valve")
    open_solenoid_valve()
    
def dehum_control_action():
    print("Turning on dehumidifier")
    turn_on_dehumidifier()
    time.sleep(HUM_CONTROL_DURATION)
    print("Turning off dehumidifier")
    turn_off_dehumidifier()

def hum_control_action():
    print("Turning on humidifier")
    turn_on_humidifier()
    time.sleep(HUM_CONTROL_DURATION)
    print("Turning off humidifier")
    turn_off_humidifier()

def temp_control_action():
    print("Turning on heater")
    turn_on_heater()
    time.sleep(TEMP_CONTROL_DURATION)
    print("Turning off heater")
    turn_off_heater()
    
#~~~~~~~~~


    
def main():
    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:
                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))
                        
                        # get HUM data
                        hum_level = float(currentReading.split(',')[0])
                        if hum_level > 0:
                            hum_readings.append(hum_level)
                            hum_queue.put(("Humidity", hum_level))
                        avg_hum = sum(hum_readings) / len(hum_readings) if hum_readings else 0
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")
                        
                        # get TEMP data
                        temp_level = float(currentReading.split(',')[1])
                        if temp_level > 0 :
                            temp_readings.append(temp_level)
                            temp_queue.put(("temperature", temp_level))
                        avg_temp = sum(temp_readings) / len(temp_readings) if temp_readings else 0
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        
                        # get CO2 data
                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_readings.append(co2_level)
                            co2_queue.put(("CO2", co2_level))
                        avg_co2 = sum(co2_readings) / len(co2_readings) if co2_readings else 0

                            
                        #co2_level = float(currentReading.split(',')[0])
                        #if co2_level > 0:
                            #co2_readings.append(co2_level)
                            #print(f"CO2 level for {device.name}: {co2_level}")
                            #print(f"Global CO2 Levels: {co2_readings}")


                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))
                    conn.commit()
                loop_counter += 1
                print(loop_counter)
                time.sleep(10)

    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()


def options():
    while True:
        print("\n--- Menu ---")
        print("1. Open solenoid valve")
        print("2. Turn on dehumidifier")
        print("3. Turn on humidifier")
        print("4. Turn on heater")
        print("5. Quit")
        choice = input("Enter your choice: ")
        
        if choice == "1":
            co2_control_action()
        elif choice == "2":
            dehum_control_action()
        elif choice == "3":
            hum_control_action()
        elif choice == "4":
            temp_control_action()
        elif choice == "5":
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 8.")

if __name__ == '__main__':
    data_queue = queue.Queue()
    device_list = detectSensors()

    main_thread = threading.Thread(target=main)
    options_thread = threading.Thread(target=options)
    

    main_thread.start()
    options_thread.start()

    main_thread.join()
    options_thread.join()

# CODE FROM PI 11/2 EDITED!

In [ ]:
try:
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    import time
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
    from multiprocessing import Process, Queue
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

# GPIO setup
GPIO_PIN = 25
PUMP_PIN = 24
PUMP_PIN_2 = 23
HEATER_PIN = 4

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN_2, GPIO.OUT)
GPIO.setup(HEATER_PIN, GPIO.OUT)

def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        #return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        #return "", "", "Unexpected error"

# CONSTANTS

N = 10

#HUM_THRESHOLD_MAX = 20 # % RH
#HUM_THRESHOLD_MIN = 10 # % RH
#TEMP_THRESHOLD = 25 # deg C
#CO2_THRESHOLD = 400

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)


PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 10
HEATER_ON_DURATION = 500

WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUM_CONTROL_INTERVAL = 45
TEMP_CONTROL_INTERVAL = 155

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

# ~~~~
# DEFINITIONS
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()

# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.05)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_on_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_on_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_off_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_off_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_on_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
        time.sleep(HEATER_ON_DURATION)
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_off_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

#~~~~~~~~

def co2_control_action():
    print("Opening CO2 valve")
    open_solenoid_valve()

def dehum_control_action():
    print("Turning on dehumidifier")
    turn_on_dehumidifier()
    time.sleep(PUMP_ON_DURATION)
    print("Turning off dehumidifier")
    turn_off_dehumidifier()

def hum_control_action():
    print("Turning on humidifier")
    turn_on_humidifier()
    time.sleep(PUMP_ON_DURATION_2)
    print("Turning off humidifier")
    turn_off_humidifier()

def temp_control_action():
    print("Turning on heater")
    turn_on_heater()
    time.sleep(HEATER_ON_DURATION)
    print("Turning off heater")
    turn_off_heater()

#~~~~~~~~~
stop_threads = False

def main():
    global stop_threads
    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while not stop_threads:
            #while True:
                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        # get HUM data
                        hum_level = float(currentReading.split(',')[0])
                        if hum_level > 0:
                            hum_readings.append(hum_level)
                            hum_queue.put(("Humidity", hum_level))
                        avg_hum = sum(hum_readings) / len(hum_readings) if hum_readings else 0
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")

                        # get TEMP data
                        temp_level = float(currentReading.split(',')[1])
                        if temp_level > 0 :
                            temp_readings.append(temp_level)
                            temp_queue.put(("temperature", temp_level))
                        avg_temp = sum(temp_readings) / len(temp_readings) if temp_readings else 0
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        # get CO2 data
                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_readings.append(co2_level)
                            co2_queue.put(("CO2", co2_level))
                        avg_co2 = sum(co2_readings) / len(co2_readings) if co2_readings else 0


                        #co2_level = float(currentReading.split(',')[0])
                        #if co2_level > 0:
                            #co2_readings.append(co2_level)
                            #print(f"CO2 level for {device.name}: {co2_level}")
                            #print(f"Global CO2 Levels: {co2_readings}")


                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))
                    conn.commit()
                loop_counter += 1
                #print(loop_counter)
                time.sleep(10)

    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()
   #pass
def options():
    global stop_threads
    while not stop_threads:
    #while True:
        try:
            print("\n--- Menu ---")
            print("1. Open solenoid valve")
            print("2. Turn on dehumidifier")
            print("3. Turn on humidifier")
            print("4. Turn on heater")
            print("5. Quit")
            choice = input("Enter your choice: ")

            if choice == "1":
                co2_control_action()
            elif choice == "2":
                dehum_control_action()
            elif choice == "3":
                hum_control_action()
            elif choice == "4":
                temp_control_action()
            elif choice == "5":
                print("Exiting menu.")
                break
            else:
                print("Invalid choice. Please enter a number from 1 to 5.")
        except Exception as e:
            print(f"An error occurred: {e}")

    #pass

def run_main():
    main_thread = threading.Thread(target=main)
    main_thread.start()
    return main_thread

def run_options():
    options_thread = threading.Thread(target=options)
    options_thread.start()
    return options_thread

if __name__ == '__main__':
    # Detect sensors before starting the threads
    device_list = detectSensors()
    if not device_list:
        print("No sensors detected. Exiting...")
        sys.exit(1)

    # Run the main and options functions in separate threads
    main_thread = run_main()
    options_thread = run_options()

    # Wait for the options thread to finish (when user selects to quit)
    options_thread.join()
    # Signal the main thread to stop
    stop_threads = True
    # Wait for the main thread to finish
    main_thread.join()

    GPIO.cleanup()

# CODE FROM PI 11/2 OG

In [ ]:
try:
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    import time
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
    from multiprocessing import Process, Queue
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

# GPIO setup
GPIO_PIN = 25
PUMP_PIN = 24
PUMP_PIN_2 = 23
HEATER_PIN = 4

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN_2, GPIO.OUT)
GPIO.setup(HEATER_PIN, GPIO.OUT)

def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        #return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        #return "", "", "Unexpected error"

# CONSTANTS

N = 10

#HUM_THRESHOLD_MAX = 20 # % RH
#HUM_THRESHOLD_MIN = 10 # % RH
#TEMP_THRESHOLD = 25 # deg C
#CO2_THRESHOLD = 400

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)


PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 10
HEATER_ON_DURATION = 500

WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUM_CONTROL_INTERVAL = 45
TEMP_CONTROL_INTERVAL = 155

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

# ~~~~
# DEFINITIONS
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()

# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.05)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_on_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_on_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_off_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_off_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_on_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
        time.sleep(HEATER_ON_DURATION)
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_off_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

#~~~~~~~~

def co2_control_action():
    print("Opening CO2 valve")
    open_solenoid_valve()

def dehum_control_action():
    print("Turning on dehumidifier")
    turn_on_dehumidifier()
    time.sleep(PUMP_ON_DURATION)
    print("Turning off dehumidifier")
    turn_off_dehumidifier()

def hum_control_action():
    print("Turning on humidifier")
    turn_on_humidifier()
    time.sleep(PUMP_ON_DURATION_2)
    print("Turning off humidifier")
    turn_off_humidifier()

def temp_control_action():
    print("Turning on heater")
    turn_on_heater()
    time.sleep(HEATER_ON_DURATION)
    print("Turning off heater")
    turn_off_heater()

#~~~~~~~~~


def main():
    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:
                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        # get HUM data
                        hum_level = float(currentReading.split(',')[0])
                        if hum_level > 0:
                            hum_readings.append(hum_level)
                            hum_queue.put(("Humidity", hum_level))
                        avg_hum = sum(hum_readings) / len(hum_readings) if hum_readings else 0
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")

                        # get TEMP data
                        temp_level = float(currentReading.split(',')[1])
                        if temp_level > 0 :
                            temp_readings.append(temp_level)
                            temp_queue.put(("temperature", temp_level))
                        avg_temp = sum(temp_readings) / len(temp_readings) if temp_readings else 0
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        # get CO2 data
                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_readings.append(co2_level)
                            co2_queue.put(("CO2", co2_level))
                        avg_co2 = sum(co2_readings) / len(co2_readings) if co2_readings else 0


                        #co2_level = float(currentReading.split(',')[0])
                        #if co2_level > 0:
                            #co2_readings.append(co2_level)
                            #print(f"CO2 level for {device.name}: {co2_level}")
                            #print(f"Global CO2 Levels: {co2_readings}")


                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))
                    conn.commit()
                loop_counter += 1
                #print(loop_counter)
                time.sleep(10)

    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()
   #pass
def options():
    while True:
        try:
            print("\n--- Menu ---")
            print("1. Open solenoid valve")
            print("2. Turn on dehumidifier")
            print("3. Turn on humidifier")
            print("4. Turn on heater")
            print("5. Quit")
            choice = input("Enter your choice: ")

            if choice == "1":
                co2_control_action()
            elif choice == "2":
                dehum_control_action()
            elif choice == "3":
                hum_control_action()
            elif choice == "4":
                temp_control_action()
            elif choice == "5":
                print("Exiting menu.")
                break
            else:
                print("Invalid choice. Please enter a number from 1 to 5.")
        except Exception as e:
            print(f"An error occurred: {e}")

    #pass

if __name__ == '__main__':
    data_queue = Queue()  # Assuming you want to use this for inter-process communication
    device_list = detectSensors()

    # Create the main process
    main_process = Process(target=main)
    # Create the options process
    options_process = Process(target=options)

    # Start both processes
    main_process.start()
    options_process.start()

    # Wait for both processes to finish
    main_process.join()
    options_process.join()

In [ ]:
try:
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    import time
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
    from multiprocessing import Process, Queue
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

# GPIO setup
GPIO_PIN = 25
PUMP_PIN = 24
PUMP_PIN_2 = 23
HEATER_PIN = 4

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.cleanup()

GPIO.setup(GPIO_PIN, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(PUMP_PIN, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(PUMP_PIN_2, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(HEATER_PIN, GPIO.OUT, initial=GPIO.LOW)

def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        #return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        #return "", "", "Unexpected error"

# CONSTANTS

N = 10

#HUM_THRESHOLD_MAX = 20 # % RH
#HUM_THRESHOLD_MIN = 10 # % RH
#TEMP_THRESHOLD = 25 # deg C
#CO2_THRESHOLD = 400

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)


PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 10
HEATER_ON_DURATION = 500

WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUM_CONTROL_INTERVAL = 45
TEMP_CONTROL_INTERVAL = 155

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

# ~~~~
# DEFINITIONS
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()

# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.05)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_on_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_on_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_off_dehumidifier():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_off_humidifier():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_on_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
        time.sleep(HEATER_ON_DURATION)
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_off_heater():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

#~~~~~~~~

def co2_control_action():
    print("Opening CO2 valve")
    open_solenoid_valve()

def dehum_control_action():
    print("Turning on dehumidifier")
    turn_on_dehumidifier()
    time.sleep(PUMP_ON_DURATION)
    print("Turning off dehumidifier")
    turn_off_dehumidifier()

def hum_control_action():
    print("Turning on humidifier")
    turn_on_humidifier()
    time.sleep(PUMP_ON_DURATION_2)
    print("Turning off humidifier")
    turn_off_humidifier()

def temp_control_action():
    print("Turning on heater")
    turn_on_heater()
    time.sleep(HEATER_ON_DURATION)
    print("Turning off heater")
    turn_off_heater()

#~~~~~~~~~

def main():
    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:
                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        # get HUM data
                        hum_level = float(currentReading.split(',')[0])
                        if hum_level > 0:
                            hum_readings.append(hum_level)
                            hum_queue.put(("Humidity", hum_level))
                        avg_hum = sum(hum_readings) / len(hum_readings) if hum_readings else 0
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")

                        # get TEMP data
                        temp_level = float(currentReading.split(',')[1])
                        if temp_level > 0 :
                            temp_readings.append(temp_level)
                            temp_queue.put(("temperature", temp_level))
                        avg_temp = sum(temp_readings) / len(temp_readings) if temp_readings else 0
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        # get CO2 data
                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_readings.append(co2_level)
                            co2_queue.put(("CO2", co2_level))
                        avg_co2 = sum(co2_readings) / len(co2_readings) if co2_readings else 0


                        #co2_level = float(currentReading.split(',')[0])
                        #if co2_level > 0:
                            #co2_readings.append(co2_level)
                            #print(f"CO2 level for {device.name}: {co2_level}")
                            #print(f"Global CO2 Levels: {co2_readings}")


                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))
                    conn.commit()
                loop_counter += 1
                #print(loop_counter)
                time.sleep(10)

    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()


def options():
    while True:
        try:
            print("\n--- Menu ---")
            print("1. Open solenoid valve")
            print("2. Turn on dehumidifier")
            print("3. Turn on humidifier")
            print("4. Turn on heater")
            print("5. Quit")
            choice = input("Enter your choice: ")

            if choice == "1":
                co2_control_action()
            elif choice == "2":
                dehum_control_action()
            elif choice == "3":
                hum_control_action()
            elif choice == "4":
                temp_control_action()
            elif choice == "5":
                print("Exiting menu.")
                break
            else:
                print("Invalid choice. Please enter a number from 1 to 5.")
        except Exception as e:
            print(f"An error occurred: {e}")

            
if __name__ == '__main__':
    # Create the main thread for polling sensors and actuation
    main_thread = threading.Thread(target=main)
    main_thread.start()

    # Run the options in the main thread
    options()

    # Wait for the main thread to finish (though your current logic may never let it finish)
    main_thread.join()
    GPIO.cleanup()
